In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[29]:


import operator
import string
from nltk import tokenize
from nltk.corpus import stopwords

# Data is a list of tweets
def getBestDressed(data, kb):
    
    result = []
       
    translator = str.maketrans('', '', string.punctuation)
    remove_terms = ['#goldenglobes', 'golden globes', '#goldenglobe', 'golden globe', 'goldenglobes', 'goldenglobe', 'golden', 'globe', 'globes']
    stop = remove_terms
    include_terms = ['dress', 'fashion', 'red', 'carpet', 'haute couture', 'gown', 'design']
    
    for tweet in data:
        
        tweet = re.sub("\d+", "", tweet)       #strip nums
        tweet = re.sub(r'http\S+', '', tweet)  #strip urls
        tweet = re.sub(r'#\S+', '', tweet)     #strip hashtags
        tweet = tweet.translate(translator)    #strip non-alphanumeric characters

        for i in stop:
            for j in tweet:
                if i.lower() in j.lower():
                    tweet.remove(j)
        
        if any(term in tweet for term in include_terms) and 'suit' not in tweet:
            result.append(tweet)
    
    
    # extract people and put in dictionary with compound scores
    sentiment_analyzer = SentimentIntensityAnalyzer()
    score_dict = {}
    best_dressed_dict = {}
    worst_dressed_dict = {}

    for tweet in result:
        all_scores = sentiment_analyzer.polarity_scores(tweet)
        for k in sorted(all_scores):
            if k == 'compound':
                useful_score = all_scores[k]

        if tweet:
            # Get all possible bigrams & trigrams in a tweet
            gram = list(nltk.everygrams(tweet.split(), 2, 3))

            # Filter through and append to list for tweet
            for g in gram:
                if len(g) == 2:
                    if bool(re.match(r'\b[A-Z][a-z]+\b', g[0])) and bool(re.match(r'\b[A-Z][a-z]+\b', g[1])):
                        name = ' '.join(g).lower()
                        if useful_score > 0:
                            if name in best_dressed_dict:
                                best_dressed_dict[name] += useful_score
                            else:
                                best_dressed_dict[name] = useful_score
                        if useful_score < 0:
                            if name in worst_dressed_dict:
                                worst_dressed_dict[name] += useful_score
                            else:
                                worst_dressed_dict[name] = useful_score
                else:
                    if bool(re.match(r'\b[A-Z][a-z]+\b', g[0])) and bool(re.match(r'\b[A-Z][a-z]+\b', g[1])) and bool(re.match(r'\b[A-Z][a-z]+\b', g[2])):
                        name = ' '.join(g).lower()
                        if useful_score > 0:
                            if name in best_dressed_dict:
                                best_dressed_dict[name] += useful_score
                            else:
                                best_dressed_dict[name] = useful_score
                        if useful_score < 0:
                            if name in worst_dressed_dict:
                                worst_dressed_dict[name] += useful_score
                            else:
                                worst_dressed_dict[name] = useful_score
        
        
    # look in kb for matches     
    final_dict = {}
    for best, worst in zip(best_dressed_dict, worst_dressed_dict):
        if best.lower() in kb:
            final_dict[best] = best_dressed_dict[best]
        if worst.lower() in kb:
            final_dict[worst] = worst_dressed_dict[worst]
            

    # get key with max value
    best_dressed = []
    worst_dressed = []
    
    while len(best_dressed) < 6:
        best_person = max(final_dict.items(), key=lambda k: k[1])[0]
        worst_person = min(final_dict.items(), key=lambda k: k[1])[0]
        best_dressed.append(best_person)
        worst_dressed.append(worst_person)
        del final_dict[best_person]
        del final_dict[worst_person]
    
    return best_dressed, worst_dressed